In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
from datetime import datetime

import tensorflow as tf

if not os.path.exists('facenet'):
  !git clone https://github.com/davidsandberg/facenet --depth=1

sys.path.insert(0, './facenet/src')
import align.detect_face
import facenet
import scipy.spatial.distance as distance

In [ ]:
model_dir = '/home/sabrine/notebook/reid/20170216-091149/'

In [ ]:
# starting a session
tf.reset_default_graph()

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.allow_soft_placement = True

sess = tf.Session(config=tf_config)

print('loading the detection/alignment network...')
pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
print('done!')

print('loading the embedding network...')
meta_file, ckpt_file = facenet.get_model_filenames(model_dir)
restorer = tf.train.import_meta_graph(os.path.join(model_dir, meta_file))
restorer.restore(sess, os.path.join(model_dir, ckpt_file))
print('done!')

# getting input / output tensors
g = tf.get_default_graph()
images_in = g.get_tensor_by_name('input:0')
phase_train_in = g.get_tensor_by_name('phase_train:0')
embeddings = g.get_tensor_by_name('embeddings:0')


In [ ]:
import cv2
import os

In [ ]:
# List all jpg file from mypath folder and store the absolute path of the first frame into ref
# ref will be an array of faces to be recognized into other images
ref = []

# detection parameters
minsize = 20 
threshold = [ 0.6, 0.7, 0.7 ]
factor = 0.709

mypath = "/home/sabrine/notebook/BIWI_dataset/Training"
for x in os.listdir(mypath):
    stored = False
    for i, f in enumerate(os.listdir(mypath + "/" + x)):
        if str(f).endswith('jpg') and not stored:   
            thePath = mypath + "/" + x + "/" + f
            img = scipy.misc.imread(thePath)
            
            bbs, kps = align.detect_face.detect_face(img, minsize, 
                                             pnet, rnet, onet, threshold, factor)
            kps = np.asarray(kps)
            kps = kps.reshape([2,5,-1]).T
            
            if len(kps) != 0:  
                ref.append(mypath + "/" + x + "/" + f)
                stored = True
                print('Number ' + x + " stored")


In [ ]:
# List all jpg file from mypath folder and store the absolute path of the first frame into ref
# ref will be an array of faces to be recognize into other images
guess = []

# detection parameters
minsize = 20 
threshold = [ 0.6, 0.7, 0.7 ]
factor = 0.709

mypath = "/home/sabrine/notebook/BIWI_dataset/Testing/Still"
for x in os.listdir(mypath):
    stored = False
    for i, f in enumerate(os.listdir(mypath + "/" + x)):
        if str(f).endswith('jpg') and not stored:   
            thePath = mypath + "/" + x + "/" + f
            img = scipy.misc.imread(thePath)
            
            bbs, kps = align.detect_face.detect_face(img, minsize, 
                                             pnet, rnet, onet, threshold, factor)
            kps = np.asarray(kps)
            kps = kps.reshape([2,5,-1]).T
            
            if len(kps) != 0:  
                guess.append(mypath + "/" + x + "/" + f)
                stored = True
                print('Number ' + x + " stored")

In [ ]:
from IPython.display import Image, display

# detection parameters
minsize = 20 
threshold = [ 0.6, 0.7, 0.7 ]
factor = 0.709
             
boxes = []
keypoints = []
images = []
print('detecting faces...')
for path in np.concatenate((guess, ref), axis=0):
    display(Image(path))
    img = scipy.misc.imread(path)
    bbs, kps = align.detect_face.detect_face(img, minsize, 
                                             pnet, rnet, onet, threshold, factor)
    kps = np.asarray(kps)
    print(kps.shape)
    kps = kps.reshape([2,5,-1]).T

    # Should always happen
    if len(kps) != 0:  
        images.append(img)  
        boxes.append(bbs)
        keypoints.append(kps)
print('done!')  

In [ ]:
print('computing embeddings')
size = 160
margin = 32

faces = []
faces_embs = []

for img, bbs in zip(images, boxes):
  img_faces = []
  for x0,y0,x1,y1,_ in bbs.astype(np.int32):
    x0 = np.maximum(x0 - margin//2, 0)
    y0 = np.maximum(y0 - margin//2, 0)
    x1 = np.minimum(x1 + margin//2, img.shape[1])
    y1 = np.minimum(y1 + margin//2, img.shape[0])
    img_faces.append(scipy.misc.imresize(img[y0:y1,x0:x1], (size, size)))
  img_faces = np.stack(img_faces)
  faces.append(img_faces)
  
  feed_dict = {
    images_in : img_faces.astype(np.float32) / 255.0,
      
    phase_train_in : False,
  }
  
  faces_embs.append(sess.run(embeddings, feed_dict))  
print('done!')

In [ ]:
type(faces[1])

In [ ]:
len(guess)

In [ ]:
len(ref)

In [ ]:
len(faces_embs)

In [ ]:

hostFaces = np.concatenate((faces[1], faces[2]), axis = 0)

In [ ]:
host = np.concatenate((faces_embs[1], faces_embs[2]), axis = 0)

In [ ]:
ref_emb = np.concatenate((faces_embs[len(guess) + 1], faces_embs[len(guess) + 2]), axis = 0)
refFaces = np.concatenate((faces[len(guess) + 1], faces[len(guess) + 2]), axis = 0)

for i in range(len(guess) + 3, len(faces_embs)):
    ref_emb = np.concatenate((ref_emb, faces_embs[i]), axis = 0)
    refFaces = np.concatenate((refFaces, faces[i]), axis = 0)

In [ ]:
guess_emb = np.concatenate((faces_embs[0], faces_embs[1]), axis = 0)
guessFaces = np.concatenate((faces[0], faces[1]), axis = 0)

for i in range(2, len(guess)):
    guess_emb = np.concatenate((guess_emb, faces_embs[i]), axis = 0)
    guessFaces = np.concatenate((guessFaces, faces[i]), axis = 0)

In [ ]:
import math
# Euclidian distance of the 2 first picture's faces
# compare each guess ambedding faces to all the ref embedding faces
dist = distance.cdist(guess_emb, ref_emb)

for i in range(len(guess_emb)):
    # idx is the arg i that minimize the euclidian distance between the face of faces_embs[1] and faces_embs[2]
    print("i " + str(i))
    idx = np.argmin(dist[i])
    print(np.min(dist[i]))
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(refFaces[j])
    plt.subplot(1,2,2)
    plt.imshow(faces[idx][0]) #[idx])
    #print(i, idx)

In [ ]:
import math
# Euclidian distance of the 2 first picture's faces
# compare each guess ambedding faces to all the ref embedding faces
for j in range(len(guess)):
    dist = distance.cdist(faces_embs[j], ref_emb)

    for i in range(1):
        # idx is the arg i that minimize the euclidian distance between the face of faces_embs[1] and faces_embs[2]
        print("i " + str(i))
        idx = np.argmin(dist[i])
        print(np.min(dist[i]))
        plt.figure()
        plt.subplot(1,2,1)
        plt.imshow(refFaces[j])
        plt.subplot(1,2,2)
        plt.imshow(faces[idx][0]) #[idx])
        #print(i, idx)

In [ ]:
plt.imshow(faces[37][0])

In [ ]:
for index in range(len(guess)):
    dist = distance.cdist(faces_embs[index], ref_emb)

    for i in range(faces_embs[0].shape[0]):
        idx = np.argmin(dist[i])
        print(np.min(dist[i]))
        plt.figure(index)
        plt.subplot(1,2,1)
        plt.imshow(faces[index][0]) #[idx])
        plt.subplot(1,2,2)
        plt.imshow(refFaces[idx])
        print(i, idx)

In [ ]:
for i in range(len(faces_embs)):
    plt.figure(i)
    plt.imshow(faces[i][0])


In [ ]:
faces[0][0].shape